In [1]:
import time

import astar
import grid
import jps
import pygame
import jps_numba

# import pyximport; pyximport.install()
import jps_c

ModuleNotFoundError: No module named 'astar'

In [2]:
def main():
    print('Эвристика Манхэттена:\n1 - вызов алгоритма A*' +
          '\n2 - вызов алгоритма JPS\n3 - вызов обоих методов\n' +
          '\nЭвристика Евклида:\n5 - вызов алгоритма A*' +
          '\n6 - вызов алгоритма JPS\n7 - вызов обоих методов\n' +
          '\nF5 - очистить сетку\n')

    while True:
        # try:
        #     size1 = int(input('Размер окна:\nШирина: '))
        #     size2 = int(input('Высота: '))
        #     size_node = int(input('Размер узла: '))
        #     if size1 % size_node != 0 or size2 % size_node != 0:
        #         print('Высота и ширина должны быть кратны размеру узла\n')
        #     else:
        #         break
        # except:
        size1, size2 = 1000, 1000
        size_node = 5
        break

    size = (size1, size2)

    pygame.init()
    screen = pygame.display.set_mode(size)
    running = True
    mark_border = False
    clear_border = False

    background = pygame.Surface(size)
    Grid = grid.Grid(size, size_node)
    Grid.refresh(background)

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    pos = pygame.mouse.get_pos()
                    if Grid.matrix[pos[1] // size_node][pos[0] // size_node] == 0:
                        mark_border = True
                    else:
                        clear_border = True
                elif event.button == 3:
                    Grid.mark_node(pygame.mouse.get_pos(), background)
            if (event.type == pygame.MOUSEBUTTONUP and
                    event.button == 1):
                mark_border = False
                clear_border = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_F5:
                    Grid.refresh(background)
                if event.key == pygame.K_1:
                    Grid.lightrefresh(background)
                    Grid.drawpath(astar.method(Grid.matrix, Grid.start, Grid.goal, 1),
                                  background, 1)
                if event.key == pygame.K_2:
                    Grid.lightrefresh(background)
                    Grid.drawpath(jps.method(Grid.matrix, Grid.start, Grid.goal, 1),
                                  background, 2)
                if event.key == pygame.K_3:
                    Grid.lightrefresh(background)
                    Grid.drawpath(astar.method(Grid.matrix, Grid.start, Grid.goal, 1),
                                  background, 1)
                    Grid.drawpath(jps.method(Grid.matrix, Grid.start, Grid.goal, 1),
                                  background, 2)
                if event.key == pygame.K_5:
                    Grid.lightrefresh(background)
                    Grid.drawpath(astar.method(Grid.matrix, Grid.start, Grid.goal, 2),
                                  background, 1)
                if event.key == pygame.K_6:
                    Grid.lightrefresh(background)
                    Grid.drawpath(jps.method(Grid.matrix, Grid.start, Grid.goal, 2),
                                  background, 2)
                if event.key == pygame.K_7:
                    m = Grid.matrix
                    Grid.lightrefresh(background)
                    # Grid.drawpath(astar.method(Grid.matrix, Grid.start, Grid.goal, 2),
                    #               background, 1)
                    Grid.drawpath(jps.method(Grid.matrix, Grid.start, Grid.goal, 1),
                                  background, 2)
                    Grid.drawpath(jps_numba.method(Grid.matrix, Grid.start, Grid.goal, 2),
                                  background, 3)
                    st = time.time()
                    p = jps_c.method(Grid.matrix, Grid.start, Grid.goal, 2)
                    Grid.drawpath((p, round(time.time() - st, 6)),
                                  background, 4)
        screen.blit(background, (0, 0))
        pygame.display.update()

        if mark_border:
            Grid.mark_border(pygame.mouse.get_pos(), background)
        if clear_border:
            Grid.clear_border(pygame.mouse.get_pos(), background)

    pygame.quit()
    return Grid

In [3]:
g = main()

Эвристика Манхэттена:
1 - вызов алгоритма A*
2 - вызов алгоритма JPS
3 - вызов обоих методов

Эвристика Евклида:
5 - вызов алгоритма A*
6 - вызов алгоритма JPS
7 - вызов обоих методов

F5 - очистить сетку

17324.0
17324.0


In [5]:
%timeit jps_c.method(g.matrix, g.start, g.goal, 2)
%timeit jps_c.method(g.matrix, g.start, g.goal, 1)

2.69 ms ± 26.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.71 ms ± 45.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit jps.method(g.matrix, g.start, g.goal, 2)
%timeit jps.method(g.matrix, g.start, g.goal, 1)

1740.3738084913084
1740.3738084913084
1740.3738084913084
1740.3738084913084
1740.3738084913084
1740.3738084913084
1740.3738084913084
1740.3738084913084
232 ms ± 4.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
17324.0
17324.0
17324.0
17324.0
17324.0
17324.0
17324.0
17324.0
234 ms ± 3.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit jps_numba.method(g.matrix, g.start, g.goal, 2)
%timeit jps_numba.method(g.matrix, g.start, g.goal, 1)

166 ms ± 1.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
163 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
import numpy as np

In [10]:
np.save('map.txt', g.matrix)

In [12]:
g.matrix = np.load('map.txt.npy')

In [14]:
%timeit jps_c.method(g.matrix, g.start, g.goal, 2)
%timeit jps_c.method(g.matrix, g.start, g.goal, 1)

2.7 ms ± 76.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.69 ms ± 29.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
